In [1]:
import json
import requests
from lxml import html
from bs4 import BeautifulSoup
from collections import OrderedDict
import argparse
import re
import dateutil.parser
import pandas as pd
import time
from selenium import webdriver
from textatistic import Textatistic
import pickle


## Pull articles list from home page

## loop through pickle list of people

In [2]:
with open("all_the_people_j.pkl", 'rb') as picklefile: 
    all_the_people = pickle.load(picklefile)


In [3]:
all_the_people

{'https://medium.com/@Mapbox',
 'https://medium.com/@MediumStaff',
 'https://medium.com/@QMlab',
 'https://medium.com/@SeattleDataGuy',
 'https://medium.com/@Sinergise',
 'https://medium.com/@acoustik',
 'https://medium.com/@agentdana',
 'https://medium.com/@ahmiglarese',
 'https://medium.com/@alecburch',
 'https://medium.com/@amycurneen',
 'https://medium.com/@andrerpena',
 'https://medium.com/@awjuliani',
 'https://medium.com/@blakelawrence001',
 'https://medium.com/@carto',
 'https://medium.com/@chad.scherrer',
 'https://medium.com/@cholmes',
 'https://medium.com/@cyrusrustomji',
 'https://medium.com/@dan.allison',
 'https://medium.com/@danmccarey',
 'https://medium.com/@djmorcode',
 'https://medium.com/@ericrodenbeck',
 'https://medium.com/@foamspace',
 'https://medium.com/@forrestbrazeal',
 'https://medium.com/@galen.ballew',
 'https://medium.com/@grega.milcinski',
 'https://medium.com/@hadleywickham',
 'https://medium.com/@ianthro',
 'https://medium.com/@james_aka_yale',
 'https:

In [7]:
people_sample = list(all_the_people)[-5:]
people_sample

['https://medium.com/@james_aka_yale',
 'https://medium.com/@cyrusrustomji',
 'https://medium.com/@danmccarey',
 'https://medium.com/@andrerpena',
 'https://medium.com/@SeattleDataGuy']

In [8]:
master_articles = []

In [9]:
def get_home_articles(soup):
    article_urls = []
    try:
        articles = soup.find_all('a', {'data-action':'open-post'})
        for item in articles:
            if item.text == 'Read more…':
                url = (item['href'])
                url = re.sub('\?source(.*)','',url)
                article_urls.append(url)
        return article_urls
    except: []

In [11]:
driver = webdriver.Chrome(executable_path="/Users/mayamidzik/tools/chromedriver")

for url in people_sample:
    url= url+'/latest'
    driver.get(url)
    
    #scroll to bottom of page
    SCROLL_PAUSE_TIME = 0.1
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    #get the html for the page
    innerHTML = driver.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,"lxml")
    
    articles = get_home_articles(soup)
    master_articles.extend(articles)

    

In [12]:
master_articles

['https://heartbeat.fritz.ai/the-5-computer-vision-techniques-that-will-change-how-you-see-the-world-1ee19334354b',
 'https://heartbeat.fritz.ai/the-5-deep-learning-frameworks-every-serious-machine-learner-should-be-familiar-with-93f4d469d24c',
 'https://medium.com/the-post-grad-survival-guide/15-systems-thinking-guidelines-to-live-in-a-world-of-uncertainty-26f62b69bee6',
 'https://medium.com/@james_aka_yale/15-systems-thinking-guidelines-to-live-in-a-world-of-uncertainty-b07d5a478e6',
 'https://medium.com/constraint-drives-creativity/70-pieces-of-wisdom-for-smart-and-driven-college-students-to-enter-the-real-world-c8a391c2eb20',
 'https://medium.com/swlh/snapchats-filters-how-computer-vision-recognizes-your-face-9ce536206fa7',
 'https://towardsdatascience.com/16-useful-advices-for-aspiring-data-scientists-6da9afa8c72c',
 'https://medium.com/@james_aka_yale/70-pieces-of-wisdom-for-smart-and-driven-college-students-to-enter-the-real-world-9fde16e62d65',
 'https://medium.com/@james_aka_y

## Original code for single page

In [500]:
#try loading page with selenium, then scrpaing
driver = webdriver.Chrome(executable_path="/Users/mayamidzik/tools/chromedriver")
url='https://medium.com/@mmidzik/latest'
driver.get(url)
print(driver.current_url)

https://medium.com/@mmidzik/latest


In [501]:
SCROLL_PAUSE_TIME = 0.1

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [502]:
innerHTML = driver.execute_script("return document.body.innerHTML")
soup = BeautifulSoup(innerHTML,"lxml")

In [503]:
#gets the href forr all the author's posts from their home page
def get_home_articles(soup):
    article_urls = []
    try:
        articles = soup.find_all('a', {'data-action':'open-post'})
        for item in articles:
            if item.text == 'Read more…':
                url = (item['href'])
                url = re.sub('\?source(.*)','',url)
                article_urls.append(url)
        return article_urls
    except: []

get_home_articles(soup)


[]

## Go into a specific article and get necessary metadata

In [364]:
#start here: https://medium.com/data-for-democracy/one-year-of-data-for-democracy-312ab48eecfd
for_DfD= 'https://medium.com/data-for-democracy/one-year-of-data-for-democracy-312ab48eecfd'
no_DfD= 'https://medium.com/@jonathonmorgan/introducing-new-knowledge-defending-public-discourse-c89f59630c3a'
tester = 'https://medium.com/sentinel-hub/improving-cloud-detection-with-machine-learning-c09dc5d7cf13'
driver.get('https://blog.mapbox.com/patagonias-dam-good-map-680d9926101f')



In [365]:
articleHTML = driver.execute_script("return document.body.innerHTML")
articlesoup = BeautifulSoup(articleHTML,"lxml")
articlesoup;

In [459]:
#check if the article is associated with a publication
def check_pub(soup):
    if soup.find_all('a',{'class':'js-collectionLogoOrName'}):
        return 1
    else: return 0

In [458]:
#get_writer

def get_author(soup):
    author_url = soup.find('a',{'data-action-source':'post_header_lockup'})
    if author_url:
        author_url = author_url['href']
        author_url = re.sub('\?source(.*)','',author_url)
        return author_url
    else: return numpy.nan

In [460]:
#get the date and time of the article:
#should write an exception in case time is missing
def get_datetime(soup):
    published = soup.find('time')
    if published: 
        return published['datetime']
    else: return numpy.nan

def get_date(soup):
    published = soup.find('time')
    if published: 
        return published.text
    else: return numpy.nan

In [461]:
#get the reading time for the article

def get_reading_time(soup):
    reading = soup.find('span',{'class':'readingTime'})
    if reading:
        reading = reading['title']
        #reading = list(filter(str.isdigit, reading))
        #return int(''.join(reading))
        return reading
    else: return numpy.nan

In [462]:
#get the tags for the article
def get_tags(soup):
    tag_list = []
    tags = articlesoup.find('ul',{'class':'tags tags--postTags tags--borderless'})
    if tags:
        tags = tags.find_all('li')
        for tag in tags:
            tag_list.append(tag.text)
    return tag_list

In [463]:
#get claps
def get_claps(soup):
    claps = articlesoup.find('button',{'data-action':'show-recommends'})
    if claps: return int(claps.text)
    else: return numpy.nan

In [479]:
article_traits = {}
currentURL =  driver.current_url
article_traits[currentURL] = {}
article_traits

def write_all_traits(soup):
    article_traits[currentURL] = {'publisher': check_pub(articlesoup)}
    article_traits[currentURL].update({'author': get_author(articlesoup)})
    article_traits[currentURL].update({'datetime': get_datetime(articlesoup)})
    article_traits[currentURL].update({'date': get_date(articlesoup)})
    article_traits[currentURL].update({'reading_time': get_reading_time(articlesoup)})
    article_traits[currentURL].update({'claps': get_claps(articlesoup)})
    return article_traits

write_all_traits(articlesoup)


{'https://blog.mapbox.com/patagonias-dam-good-map-680d9926101f': {'author': 'https://blog.mapbox.com/@Mapbox',
  'claps': 1,
  'date': 'Apr 18',
  'datetime': '2018-04-18T20:20:20.291Z',
  'publisher': 1,
  'reading_time': '2 min read'}}

In [480]:
#get all attributes around the article text
article_text_traits= {}

def get_text_traits(soup):
    
    #get the number of images and add to dictionary
    article_images = soup.find_all('img',{'class' : 'progressiveMedia-image js-progressiveMedia-image'})
    if article_images:
        #add to dictionary
        article_text_traits['images'] = len(article_images)
    else: article_text_traits['images'] = np.nan
    
    #get the tota; text block for the article
    article_text = articlesoup.find_all('p')
    total_text = ''
    if article_text:
        for text in article_text:
            total_text+=(text.text)
            
        #get total word count
        article_text_traits['words'] = len(total_text.split())
        
        #get sentence count
        article_text_traits['sentences'] = total_text.count('.')
        
        #get flesch readability
        try:
            s = Textatistic(total_text)
            article_text_traits['flesch'] = s.flesch_score
        except: article_text_traits['flesch'] = np.nan
            
    else:
        article_text_traits['words'] = np.nan
        article_text_traits['sentences'] = np.nan
        article_text_traits['flesch'] = np.nan
        

get_text_traits(articlesoup)
article_text_traits


{'flesch': 55.737517838939894, 'images': 5, 'sentences': 9, 'words': 216}

In [481]:
article_traits[currentURL].update(article_text_traits)
article_traits

{'https://blog.mapbox.com/patagonias-dam-good-map-680d9926101f': {'author': 'https://blog.mapbox.com/@Mapbox',
  'claps': 1,
  'date': 'Apr 18',
  'datetime': '2018-04-18T20:20:20.291Z',
  'flesch': 55.737517838939894,
  'images': 5,
  'publisher': 1,
  'reading_time': '2 min read',
  'sentences': 9,
  'words': 216}}

In [484]:
#write to a dataframe
df = pd.DataFrame.from_dict(article_traits).T
df.reset_index(level=0, inplace=True)
df['index'] = df
df.rename(columns = {'index':'url'},inplace = True)
df.head()

,url,author,claps,date,datetime,flesch,images,publisher,reading_time,sentences,words
0,https://blog.mapbox.com/patagonias-dam-good-ma...,https://blog.mapbox.com/@Mapbox,1,Apr 18,2018-04-18T20:20:20.291Z,55.7375,5,1,2 min read,9,216
